# CSCI 532 Semester Project Demonstration and Analysis &mdash; Go Version

James Avery

## Problem #3

>Write a C/C++/Matlab/Java program to compute insertion sort and merge sort. (You can use the code from the textbooks by Deitel and Deitel or from the web.) Obtain the run time of both routines.

>The input data should be an int array containing random element values (between, say, 0 and 1023). Obtain run time T with 1D (input) array of size of $n = 16$, $256$, $4096$, $65536$, $1048576$ (i.e. $2^p$, where $p = 4$, $8$, $12$, $16$, $20$). The run time for each $n$ should be averaged with about $\left\lfloor{\frac{512}{p \times p}}\right\rfloor$ runs. Each run for a given $n$ should use a different random input.

>Plot (with Excel, Matlab, or other available tools) the run time for both routines on one plot, with the $x$ axis in $p$ values, and $y$ axis in $\log{T}$. Label on the plot which curve is for insertion sort and which is for merge sort.

>Submit C/C++ programs and plot, with instructions in readme.txt on how to build and run the program. (Include the Dev-C++/MS Visual Studio/Java NetBeans project file.)

In addition to the above, I also added a number of command line arguments to make testing more interactive:
* `insertion-only`: Prevents merge sort from being run
* `merge-only`: Prevents insertion sort from being run
* `equal-batches`: Instead of running each sort $\left\lfloor\frac{512}{p\times{p}}\right\rfloor$ times, run in equal-sized batches. This takes a lot longer, but allows for more accurate analysis, as well as processing run-time data in Python using NumPy
* `limit-n`: Limits the size of arrays to those that run quickly, i.e., arrays smaller than around 1 million items. This speeds testing the algorithms' run-times, especially when `equal-batches` is also used.
* `demo-sorters`: Demonstrates the sorting routines (specifically that they actually do sort arrays).
* `no-time`: Does not obtain run-times for the sorting algorithms (i.e. does not do what the assignment requires, in order to shorten run-times for `demo-sorters`.)

In initial testing, I noticed that the C++ version, based on Deitel and Deitel's code, was *extremely* slow due to the use of C++11 `std::vector`s rather than C-style arrays. I researched how to parallelize code using C++11's `std::thread` API and the parallel merge sort routine in Cormen et al chapter 27, however the overhead of creating extra threads slowed the program even further and crashed sorting arrays of size $n = 2^{20}$. A better option would be to use `std::async` and futures to serially sort multiple vectors in parallel, but I was not able to grasp that API in time for the project.

I had read before that Google's Go language has native support for concurrency, and after a few hours with the Go tutorials I decided to attempt to reimplement the project in Go. Converting the non-parallelized C++ code was straightforward, and I was able to redo several days' worth of C++ work in a few hours in Go. Go's goroutines and channels make implementing concurrent code reasonably simple. Furthermore, operations on Go's built-in array slice data structure (based on C-style arrays) are much faster than C++ `std::vector`s. Serial `slice`-based Go code ran an order of magnitude faster than `std::vector`-based C++ code, and parallelizing the execution further cut run-times approximately in half.

## Build the project

The Go language version of the project is built with the command `go build` in a terminal or Windows PowerShell.

In Linux: `go build -o gomain main.go`

In Windows: `go build -o gomain.exe main.go`

The code cell below checks to see if the executable exists, and builds it if it does not.

In [ ]:
import os
import platform

linux = platform.system() == "Linux"
windows = platform.system() == "Windows"

gomain_exists = os.access("gomain", os.F_OK)
gomainexe_exists = os.access("gomain", os.F_OK)

gomain_is_executable = os.access("gomain", os.X_OK)
gomainexe_is_executable = os.access("gomain.exe", os.X_OK)

if linux and gomain_is_executable:
    print("Executable exists for Linux.")
elif linux and not gomain_is_executable and not gomain_exists:
    print("Building for Linux")
    !go build -o gomain main.go
    print("Done.")
elif windows and gomainexe_is_executable:
    print("Executable exists for Windows.")
elif windows and not gomainexe_is_executable and not gomainexe_exists:
    print("Building for Windows")
    !go build -o gomain.exe main.go
    print("Done.")
else:
    print("Either this is an unsupported OS, or something else is wrong.")

## Demonstrate that the sorters sort correctly

Using a combination of the `demo-sorters` and `no-time` command line arguments, it is possible to demonstrate that the insertion sort and merge sort functions do indeed sort arrays into nondecreasing order.

### Insertion Sort

In [ ]:
if linux:
    ! ./gomain demo-sorters insertion-only no-time
elif windows:
    ! gomain.exe demo-sorters merge-only no-time

### Merge Sort

In [ ]:
if linux:
    ! ./gomain demo-sorters merge-only no-time
elif windows:
    ! gomain.exe demo-sorters merge-only no-time

## Find Run-Times

This is the main part of project as assigned, with one difference: for the analysis portion I run the sorters with the `equal-batches` flag, in order to use NumPy to analyze the data. The total run time to sort 24 vectors of 5 sizes using both algorithms is around 45 minutes on an i7-8700 with 16 GB of RAM.

### Insertion Sort

In [ ]:
if linux:
    !./gomain insertion-only equal-batches
elif windows:
    ! gomain.exe insertion-only equal-batches

### Merge Sort

In [ ]:
if linux:
    !./gomain merge-only equal-batches
elif windows:
    ! gomain.exe insertion-only equal-batches

## Prepare the data for processing

Load the run-times into numpy arrays for processing. (Note that NumPy requires each row of an array to be the same length, so this only works if the program was run with `equal-batches` as above.)

In [ ]:
import numpy as np

insertion = np.loadtxt(
    './insertionsort.csv', dtype='uint64', delimiter=',')
merge = np.loadtxt(
    './mergesort.csv', dtype='uint64', delimiter=',')

The data is stored in rows of the format $n, r_1, r_2, \ldots r_i$, where $n$ is the size of the array, and $r_i$ is the run-time in nanoseconds of the $i^{\textrm{th}}$ run, so a little data muging is necessary. The first column becomes the labels, and the subsequent columns are retained as data.

In [ ]:
insertion_labels = insertion[:, 0]
insertion = insertion[:, 1:]
merge_labels = merge[:, 0]
merge = merge[:, 1:]

The `*_labels` arrays *should* contain exactly the same data. If they don't, something has gone wrong. If they do, then it's safe to remove one of them and just use the other for all labels.

In [ ]:
if not np.all(insertion_labels == merge_labels):
    raise ValueError(
        "The two data sets were not tested on arrays of the same sizes." +
            "\n\tInsertion Sort was tested with arrays of sizes:\n\t\t" +
            str(insertion_labels) +
            "\n\tMerge Sort was tested with arrays of sizes:\n\t\t" +
            str(merge_labels))
else:
    labels = insertion_labels.copy()
    del merge_labels, insertion_labels

Find the mean run time for each vector size and each algorithm.

In [ ]:
insertion_mean, merge_mean = insertion.mean(axis=1), merge.mean(axis=1)

## Graph the results

In [ ]:
import matplotlib.pyplot as plt
import IPython

plt.rcParams['figure.figsize'] = [10, 10]
plt.plot(labels, insertion_mean, label='Insertion Sort')
plt.plot(labels, merge_mean, label='Merge Sort')
plt.legend()
plt.xscale('log')
plt.yscale('log')
plt.xlabel("Array size $n$")
plt.ylabel('Run Time (ns)')
plt.title("Average Run Times of Insertion Sort and Merge Sort Compared")

table = "| $n$ | Insertion Sort | Merge Sort |\n|:---|:---:|:---:|\n"
for i in range(len(labels)):
    table += "| {n} | {ins:.5} | {ms:.5} |\n".format(
        n=labels[i], 
        ins=insertion_mean[i], 
        ms=merge_mean[i]
    )
IPython.display.Markdown(table)